# Compare different Results from different servers

1. This notebook will target the directories `coffeelake1`, `alder`, and `bombay`

2. The files needed inside each directory are `All_Optimizations-<hostanme>.csv` and `All_OptimizationsSYCL-CUDA-<hostname>.csv`

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
from common.utils import get_best_optimization

# list of hosts
host_list = ['coffeelake1', 'alder', 'bombay']

tbb_res = {}
sycl_res = {}

# get all the results
for hostname in host_list:
    # get the TBB results
    df = pd.read_csv(os.path.join(hostname, f'All_Optimizations-{hostname}.csv'), sep=';')
    # get Total times
    df.insert(4,"Total",0)
    df['Total']=df['TimeTree']+df['TimeOWM']
    # get the best optimization
    _,besttbb_label = get_best_optimization(df)
    # keep only Baseline and the best optimization
    df = df.loc[df['Optimization'].isin(['Baseline', besttbb_label])]
    df.reset_index(drop=True, inplace=True)
    # keep the dataframe
    tbb_res[hostname] = df.copy()

    # get SYCL results
    dfs=pd.read_csv(os.path.join(hostname, f'All_OptimizationsSYCL-CUDA-{hostname}.csv'), sep=';')
    # get the Total times
    dfs.insert(4,"Total",0)
    dfs['Total']=dfs['TimeTree']+dfs['TimeOWM']
    # keep only memo versions
    dfm = dfs[~dfs["Optimization"].str.contains("nomemo")].copy()
    # keep the best version between CUDA and CUDA-grid
    opts = dfm['Optimization'].unique()
    if 'owm-cuda' in opts and 'owm-cuda-grid' in opts:
        if dfm.loc[dfm['Optimization'] == 'owm-cuda', 'Total'].mean() > dfm.loc[dfm['Optimization'] == 'owm-cuda-grid', 'Total'].mean():
            # drop CUDA without grid
            drop_label = 'owm-cuda'
        else:
            drop_label = 'owm-cuda-grid'
        # drop the selected label
        dfm = dfm[~dfm['Optimization'].isin([drop_label])].copy()
    # keep the dataframe
    sycl_res[hostname] = dfm.copy()
display(sycl_res)

# Compare each implementation against the others hosts

In [ ]:
# suppose coffeelake1 as the reference
tbb_ref,_ = get_best_optimization(tbb_res['coffeelake1'])

for hostname in tbb_res:
    if hostname == 'coffeelake1':
        continue
    # get the best optimization
    besttbb,besttbb_label = get_best_optimization(tbb_res[hostname])
    ref = np.array(tbb_ref)
    best = np.array(besttbb)
    speedupTBB = ref/best
    print(f"Speedup O4 {hostname} against coffeelake1:")
    print("Tree C., OWM Trav., Total")
    print(speedupTBB)


# Time figures

In [ ]:
import seaborn as sns

def plot_tbb_comparison(res,xlab):
    """Plot the times of the optimizations pointed by the xlab list.

    Parameters
    ----------
    res
        dictinoary with the dataframes of the results
    xlab
        list of the labels of the optimizations to plot
    """
    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-']

    # get the list of clouds
    clouds=['Alcoy','Arzua','BrionF','BrionU']
    phases=['OWM Trav.','Tree Const.']

    # get a color palette
    palette = sns.color_palette("deep")

    # generate our own dictonary with the optimizations in xlab
    plotres = []
    for hostname in res:
        # point to the dataframe
        df = res[hostname]
        # get the times in miliseconds
        times = np.array(df.loc[df['Optimization'].str.contains('|'.join(xlab)), 'TimeTree':'TimeOWM'])*1000

        # concatenate the times in a single array
        if len(plotres) == 0:
            plotres = times
        else:
            plotres = np.concatenate((plotres, times), axis=1)
    
    print(plotres)

    x=np.arange(1,len(host_list)+1)
    width=0.5
    #define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(15, 5), constrained_layout=True)#, sharey=True)
    for i,name in enumerate(clouds):
        b=np.zeros(len(host_list))
        for j,z in zip(phases,reversed(range(len(phases)))): #reversed to have OWM first
            axs[i].bar(x, plotres[i][z::2], width, label=j, bottom=b, color=palette[z])
            b=b+plotres[i][z::2]    

        axs[i].set_title(name,fontsize=16)
        axs[i].set_xlabel('Host', fontsize=xlabelfs)
        axs[i].set_xticks(x,labels=['Coffee','Alder','Bombay'],fontsize=xticksfs)
        # axs[i].yticks(fontsize=yticksfs)
        axs[i].grid()
    fig.suptitle(f'TBB best execution times (ms.) for each host',  fontweight='bold', fontsize=18)
    
    axs[0].set_ylabel('Time in milliseconds', fontsize=ylabelfs)
    axs[0].legend(loc='best', fontsize= 14)
    pp = PdfPages(f"Best-time-TBB-cross-platform.pdf")
    pp.savefig(fig)
    pp.close()

plot_tbb_comparison(tbb_res, ['MinRad'])

In [ ]:
import seaborn as sns

def plot_sycl_comparison(res, vlabels):
    """Plot the times of the optimizations pointed by the xlab list.

    Parameters
    ----------
    res
        dictinoary with the dataframes of the results
    xlab
        list of the labels of the optimizations to plot
    """
    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 14
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-']

    # get the list of clouds
    clouds=['Alcoy','Arzua','BrionF','BrionU']
    phases=['OWM Trav.','Tree Const.']

    # get a color palette
    palette = sns.color_palette("deep")

    # labels colors; CUDA green, and INTEL blue
    labels_colors = ['#76B900', '#76B900', '#00C7FD']

    # generate our own dictonary with the optimizations in xlab
    plotres = []
    for hostname in res:
        # get the best optimization
        df, best_label = get_best_optimization(res[hostname])
        print(f"Best optimization for {hostname}: {best_label}")
        # get the times in miliseconds
        times = np.array(df.loc[:,'TimeTree':'TimeOWM'])*1000

        # concatenate the times in a single array
        if len(plotres) == 0:
            plotres = times
        else:
            plotres = np.concatenate((plotres, times), axis=1)
    
    print(plotres)

    x=np.arange(1,len(host_list)+1)
    width=0.5
    #define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(15, 5), constrained_layout=True)#, sharey=True)
    for i,name in enumerate(clouds):
        b=np.zeros(len(host_list))
        for j,z in zip(phases,reversed(range(len(phases)))): #reversed to have OWM first
            axs[i].bar(x, plotres[i][z::2], width, label=j, bottom=b, color=palette[z])
            b=b+plotres[i][z::2]    

        # print a label on top of the bars
        for j, v in enumerate(vlabels):
            axs[i].text(x[j], (plotres[i][0::2][j]+plotres[i][1::2][j])/2, f'{v}', ha='center', va='center', fontsize=xlabelfs-2, fontweight='bold', color='k', rotation=90)

        axs[i].set_title(name,fontsize=16)
        axs[i].set_xlabel('Host', fontsize=xlabelfs)
        axs[i].set_xticks(x,labels=['Coffee','Alder','Bombay'],fontsize=xticksfs)
        # axs[i].yticks(fontsize=yticksfs)
        axs[i].grid()
    fig.suptitle(f'SYCL best execution times (ms.) for each host',  fontweight='bold', fontsize=18)
    
    axs[0].set_ylabel('Time in milliseconds', fontsize=ylabelfs)
    axs[0].legend(loc='best', fontsize= 14)
    pp = PdfPages(f"Best-time-SYCL-cross-platform.pdf")
    pp.savefig(fig)
    pp.close()

plot_sycl_comparison(sycl_res, ['CUDA','S-dGPU','S-CPU'])